In [2]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta


In [5]:
# Re-importing the test.csv dataset using the correct date column name

# Path = 'test4.csv'
# assets = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']
assets = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','DIS','CAT','XOM','CVX','RIO','BHP']
asset = 'AAPL'
Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/5_MTL_GARCH_{}.csv'.format(asset)
# Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
# Path = './Evaluation/GCH_1.csv'
GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
GARCH_mpt 

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-28,0.035281,0.037336,0.040123,0.037863,0.022661
2008-01-04,0.038265,0.040072,0.035695,0.042627,0.086782
2008-01-11,0.031504,0.036611,0.042072,0.040041,0.086086
2008-01-18,0.031219,0.041058,0.038230,0.043472,0.095092
2008-01-25,0.050572,0.033109,0.038630,0.040718,0.077436
...,...,...,...,...,...
2015-11-27,0.023853,0.024382,0.022982,0.037138,0.015611
2015-12-04,0.028164,0.028573,0.020585,0.019179,0.042410
2015-12-11,0.017670,0.028194,0.021271,0.017837,0.030305


In [30]:
# Load the updated data
df_updated = GARCH_mpt.copy()
df_updated 
def MASE(df_updated):
    # Calculate MAE for each horizon
    mae_h1_updated = np.mean(np.abs(df_updated['h1'] - df_updated['TrueSD']))
    mae_h2_updated = np.mean(np.abs(df_updated['h2'] - df_updated['TrueSD'].shift(1))[1:])
    mae_h3_updated = np.mean(np.abs(df_updated['h3'] - df_updated['TrueSD'].shift(2))[2:])
    mae_h4_updated = np.mean(np.abs(df_updated['h4'] - df_updated['TrueSD'].shift(3))[3:])

    # Calculate MAE of the naive forecast (using last known value)
    mae_naive_updated = np.mean(np.abs(df_updated['TrueSD'].shift(1) - df_updated['TrueSD'])[1:])
    mae_naive_updated_2 = np.mean(np.abs(df_updated['TrueSD'].shift(2) - df_updated['TrueSD'])[2:])
    mae_naive_updated_3 = np.mean(np.abs(df_updated['TrueSD'].shift(3) - df_updated['TrueSD'])[3:])
    mae_naive_updated_4 = np.mean(np.abs(df_updated['TrueSD'].shift(4) - df_updated['TrueSD'])[4:])

    # Compute MASE for each horizon
    mase_h1_updated = mae_h1_updated / mae_naive_updated
    mase_h2_updated = mae_h2_updated / mae_naive_updated_2
    mase_h3_updated = mae_h3_updated / mae_naive_updated_3
    mase_h4_updated = mae_h4_updated / mae_naive_updated_4

    return mase_h1_updated, mase_h2_updated, mase_h3_updated, mase_h4_updated

MASE(df_updated)

(1.3992542577330904,
 1.2023828181090996,
 1.0510969463627993,
 1.0101785862183106)

In [31]:
assets = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','DIS','CAT','XOM','CVX','RIO','BHP']
Results=pd.DataFrame({'Asset':[], 'h1': [], 'h2': [], 'h3':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/5_MTL_GARCH_{}.csv'.format(asset)
    # Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    IterResults={'Asset': asset, 'h1': MASE(GARCH_mpt)[0], 'h2': MASE(GARCH_mpt)[1], 'h3':MASE(GARCH_mpt)[2], 'h4': MASE(GARCH_mpt)[3]}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)    
    

In [32]:
Results

,Asset,h1,h2,h3,h4
0,AAPL,1.315179,1.098823,1.016421,0.978000
1,MSFT,1.304855,1.080413,0.946955,0.958347
2,NVDA,1.122250,0.968656,0.877225,0.941153
3,JNJ,2.071343,1.573854,1.443756,1.327749
4,NVS,1.573278,1.354038,1.223347,1.173647
5,JPM,1.524961,1.214090,1.186630,0.996711
6,GS,1.431709,1.156844,1.004914,1.038612
7,DIS,1.701791,1.227618,1.195141,1.143329
8,CAT,1.420249,1.142888,1.012541,0.997900
9,XOM,1.638329,1.278039,1.121019,1.091685


In [33]:
Assest = ['AAPL' ,'MSFT' ,'NVDA', 'JNJ', 'NVS' ,'JPM' ,'GS' ,'AMZN']
mask = Results['Asset'].isin(Assest)
Result = Results[mask]
Result.T

,0,1,2,3,4,5,6
Asset,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS
h1,1.31518,1.30485,1.12225,2.07134,1.57328,1.52496,1.43171
h2,1.09882,1.08041,0.968656,1.57385,1.35404,1.21409,1.15684
h3,1.01642,0.946955,0.877225,1.44376,1.22335,1.18663,1.00491
h4,0.978,0.958347,0.941153,1.32775,1.17365,0.996711,1.03861


In [134]:
Result.T

,0,1,2,3,4,5,6,7
Asset,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN
h1,1.29788,1.43345,1.14386,2.17157,1.54091,1.47942,1.49694,1.01041
h2,1.06223,1.10547,0.997428,1.66882,1.35683,1.17702,1.17347,0.975521
h3,1.04158,1.01485,0.879377,1.53207,1.31019,1.05726,1.02156,0.859923
h4,0.953383,0.969804,0.956979,1.44224,1.32279,0.923506,0.945403,0.885935


## RMSE

In [8]:
import numpy as np

def RMSE(actual, predicted):
    """
    Calculate the Root Mean Square Error (RMSE) between actual and predicted values.
    
    Parameters:
    - actual (array-like): The true values.
    - predicted (array-like): The predicted values.
    
    Returns:
    - Root Mean Square Error (RMSE) value.
    """
    return round(np.sqrt(np.mean((actual - predicted)**2)),5)*100


In [15]:
assets = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','DIS','CAT','XOM','CVX','RIO','BHP']
Results=pd.DataFrame({'Asset':[], 'h1': [], 'h2': [], 'h3':[], 'h4': []})
for asset in assets:
    # Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/5_MTL_GARCH_{}.csv'.format(asset)
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    IterResults={'Asset': asset, 'h1': RMSE(GARCH_mpt['TrueSD'],GARCH_mpt['h1']), 'h2':RMSE(GARCH_mpt['TrueSD'].shift(-1),GARCH_mpt['h2']), 'h3':RMSE(GARCH_mpt['TrueSD'].shift(-2),GARCH_mpt['h3']), 'h4': RMSE(GARCH_mpt['TrueSD'].shift(-3),GARCH_mpt['h4'])}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)    

In [16]:
Results.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
Asset,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,DIS,CAT,XOM,CVX,RIO,BHP
h1,21.976,36.988,60.666,3.99,10.633,178.643,67.354,19.491,31.48,7.429,19.075,86.1,43.858
h2,21.944,37.627,61.438,3.475,11.003,181.002,68.179,19.969,31.467,7.418,16.316,87.326,44.512
h3,22.04,38.208,61.937,3.634,11.38,183.553,68.918,20.437,31.699,7.5,14.993,88.461,45.163
h4,22.204,38.713,62.376,3.849,11.706,185.796,69.683,20.898,32.042,7.584,14.471,89.489,45.717


### MAPE

In [17]:
def MAPE(actual, predicted):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between actual and predicted values.
    
    Parameters:
    - actual (array-like): The true values.
    - predicted (array-like): The predicted values.
    
    Returns:
    - Mean Absolute Percentage Error (MAPE) value.
    """
    # Avoid division by zero by adding a small value to the denominator
    return round(np.mean(np.abs((actual - predicted) / (actual + 1e-10))) * 100,2)




In [19]:
assets = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','DIS','CAT','XOM','CVX','RIO','BHP']
Results=pd.DataFrame({'Asset':[], 'h1': [], 'h2': [], 'h3':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/5_MTL_GARCH_{}.csv'.format(asset)
    # Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    IterResults={'Asset': asset, 'h1': MAPE(GARCH_mpt['TrueSD'],GARCH_mpt['h1']), 'h2':MAPE(GARCH_mpt['TrueSD'].shift(-1),GARCH_mpt['h2']), 'h3':MAPE(GARCH_mpt['TrueSD'].shift(-2),GARCH_mpt['h3']), 'h4': MAPE(GARCH_mpt['TrueSD'].shift(-3),GARCH_mpt['h4'])}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)   
Results.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
Asset,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,DIS,CAT,XOM,CVX,RIO,BHP
h1,49.34,56.85,46.16,102,65.06,61.18,54.43,59.83,56.42,68.58,64.32,40.14,42.95
h2,49.41,58.74,43.4,102.42,67.71,63,56.37,55.48,56.36,68.61,66.97,41.63,46.63
h3,52.5,57.19,45.47,108.18,69.59,61.3,54.8,63.33,58.43,67.83,64.46,39.68,45.5
h4,51.1,58.49,48,98.73,65.74,62.49,58.01,60.1,56.64,68.76,70.2,43.53,44.83


In [20]:
GARCH_mpt

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-28,0.040638,0.031653,0.044669,0.039509,0.037460
2008-01-04,0.040739,0.040296,0.037721,0.039729,0.056285
2008-01-11,0.037749,0.064736,0.064750,0.069899,0.082634
2008-01-18,0.043178,0.040111,0.038850,0.043517,0.084516
2008-01-25,0.038826,0.035312,0.034107,0.040749,0.037792
...,...,...,...,...,...
2015-11-27,0.042745,0.043717,0.037353,0.047636,0.050692
2015-12-04,0.032516,0.034407,0.040991,0.047812,0.054383
2015-12-11,0.024879,0.030670,0.024024,0.033002,0.071654


In [21]:
RMSE(GARCH_mpt['TrueSD'],GARCH_mpt['h1'])

3.1649999999999996

# Aggregate Monthly

In [22]:
# Convert the index to datetime format

# Resample to monthly frequency
df_monthly = df_updated.resample('M').median()

df_monthly.head()

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-31,0.035281,0.037336,0.040123,0.037863,0.022661
2008-01-31,0.034885,0.038342,0.038430,0.041673,0.086434
2008-02-29,0.042320,0.047387,0.038093,0.039017,0.056049
2008-03-31,0.040864,0.040034,0.036513,0.036358,0.064769
2008-04-30,0.038153,0.036135,0.040147,0.036175,0.056904


In [23]:
# Adjust the compute_monthly_mase function to use a constant naive forecast for each horizon
def compute_direct_monthly_mase(row, forecast_col, actual_col='TrueSD', naive_forecast_col='naive_forecast'):
    """
    Compute MASE for a given month using a specific forecast column for direct multi-step forecasting.
    """
    mae = np.abs(row[forecast_col] - row[actual_col])
    mae_naive = np.abs(row[naive_forecast_col] - row[actual_col])
    return mae / mae_naive

# Add a column for naive forecast (previous month's TrueSD value)
df_monthly['naive_forecast'] = df_monthly['TrueSD'].shift(1)

# Compute monthly MASE for each forecast horizon using the adjusted function
mase_h1_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h1', axis=1)
mase_h2_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h2', axis=1)
mase_h3_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h3', axis=1)
mase_h4_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h4', axis=1)

# Combine into a DataFrame for better visualization
mase_direct_monthly_df = pd.DataFrame({
    'MASE_h1': mase_h1_direct_monthly,
    'MASE_h2': mase_h2_direct_monthly,
    'MASE_h3': mase_h3_direct_monthly,
    'MASE_h4': mase_h4_direct_monthly
})

mase_direct_monthly_df


,MASE_h1,MASE_h2,MASE_h3,MASE_h4
Date_Forecast,,,,
2007-12-31,NaN,NaN,NaN,NaN
2008-01-31,0.808331,0.754120,0.752737,0.701888
2008-02-29,0.451847,0.285073,0.590961,0.560536
2008-03-31,2.741481,2.836664,3.240437,3.258282
2008-04-30,2.384183,2.640851,2.130694,2.635765
...,...,...,...,...
2015-08-31,0.718004,1.015812,0.730531,0.788057
2015-09-30,0.237539,0.186262,0.126441,0.298679
2015-10-31,2.182077,0.960204,6.316014,5.830988


In [34]:
np.mean(mase_direct_monthly_df)

MASE_h1    3.578517
MASE_h2    4.200966
MASE_h3    3.951750
MASE_h4    4.006468
dtype: float64

In [25]:
mase_direct_monthly_df

,MASE_h1,MASE_h2,MASE_h3,MASE_h4
Date_Forecast,,,,
2007-12-31,NaN,NaN,NaN,NaN
2008-01-31,0.808331,0.754120,0.752737,0.701888
2008-02-29,0.451847,0.285073,0.590961,0.560536
2008-03-31,2.741481,2.836664,3.240437,3.258282
2008-04-30,2.384183,2.640851,2.130694,2.635765
...,...,...,...,...
2015-08-31,0.718004,1.015812,0.730531,0.788057
2015-09-30,0.237539,0.186262,0.126441,0.298679
2015-10-31,2.182077,0.960204,6.316014,5.830988


In [26]:
import plotly.graph_objects as go

# Create the interactive line plot using Plotly
fig = go.Figure()

# Add lines for each forecast horizon with different markers
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h1'], mode='lines+markers', name='MASE_h1', marker=dict(symbol='circle')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h2'], mode='lines+markers', name='MASE_h2', marker=dict(symbol='square')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h3'], mode='lines+markers', name='MASE_h3', marker=dict(symbol='diamond')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h4'], mode='lines+markers', name='MASE_h4', marker=dict(symbol='cross')))

# Add a horizontal line at MASE=1 for reference
fig.add_shape(
    type='line',
    line=dict(dash='dash', color='black'),
    x0=mase_direct_monthly_df.index[0],
    x1=mase_direct_monthly_df.index[-1],
    y0=1,
    y1=1
)

# Update the layout for transparent background, black fonts, and other stylistic choices
fig.update_layout(
    title='MASE values for Different Forecast Horizons over Time',
    xaxis_title='Date',
    yaxis_title='MASE',
    template="none",
    yaxis=dict(type='log', gridcolor='gray', tickcolor='gray', tickfont=dict(color='black', size=12)),
    xaxis=dict(gridcolor='gray', tickcolor='gray', tickfont=dict(color='black', size=12)),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent background for the plot area
    paper_bgcolor='rgba(0,0,0,0)',   # Transparent background for the whole figure
    font=dict(color='black', size=12, family='Sans Serif'),
    legend=dict(font=dict(color='black', size=15, family='Sans Serif'))
)

fig.show()


In [27]:
import plotly.graph_objects as go

# Create the interactive line plot using Plotly
fig = go.Figure()

# Add lines for each forecast horizon with different markers
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h1'], mode='lines+markers', name='MASE_h1', marker=dict(symbol='circle')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h2'], mode='lines+markers', name='MASE_h2', marker=dict(symbol='square')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h3'], mode='lines+markers', name='MASE_h3', marker=dict(symbol='diamond')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h4'], mode='lines+markers', name='MASE_h4', marker=dict(symbol='cross')))

# Add a horizontal line at MASE=1 for reference
fig.add_shape(
    type='line',
    line=dict(dash='dash', color='white'),
    x0=mase_direct_monthly_df.index[0],
    x1=mase_direct_monthly_df.index[-1],
    y0=1,
    y1=1
)

# Update the layout for transparent background and other stylistic choices
fig.update_layout(
    title='MASE values for Different Forecast Horizons over Time',
    xaxis_title='Date',
    yaxis_title='MASE',
    template="none",
    yaxis=dict(type='log', gridcolor='gray', tickcolor='gray', tickfont=dict(color='black')),
    xaxis=dict(gridcolor='gray', tickcolor='gray', tickfont=dict(color='black')),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent background for the plot area
    paper_bgcolor='rgba(0,0,0,0)',   # Transparent background for the whole figure
    font=dict(color='black'),
    legend=dict(font=dict(color='black'))
)

fig.show()


In [28]:
import pandas as pd
from scipy.stats import skew, kurtosis
import numpy as np

def round_to_sf(number, sf):
    """Round number to specified significant figures."""
    return round(number, sf - int(np.floor(np.log10(abs(number)))) - 1)

def compute_summary_statistics(stock_data):
    """
    Compute summary statistics for weekly returns of multiple stocks.

    Parameters:
    - stock_data: DataFrame where columns are stock tickers and rows represent weekly returns.

    Returns:
    - summary_df: DataFrame with summary statistics for each stock.
    """
    # Initializing an empty DataFrame for summary statistics
    summary_df = pd.DataFrame(index=['Min', 'Max', 'Mean', 'Std', 'Skewness', 'Kurtosis'])

    # Looping through each stock and computing the statistics
    for stock in stock_data.columns:
        stock_returns = stock_data[stock].dropna()  # Remove NaN values
        stats = {
            'Min': round_to_sf(stock_returns.min(), 4),
            'Max': round_to_sf(stock_returns.max(), 4),
            'Mean': round_to_sf(stock_returns.mean(), 4),
            'Std': round_to_sf(stock_returns.std(), 4),
            'Skewness': round_to_sf(skew(stock_returns), 4),
            'Kurtosis': round_to_sf(kurtosis(stock_returns), 4)
        }
        summary_df[stock] = stats.values()

    return summary_df

# Example usage:
# Assuming 'weekly_returns_df' is your DataFrame containing weekly returns for multiple stocks
# summary_statistics = compute_summary_statistics(weekly_returns_df)
# print(summary_statistics)


In [29]:
import yfinance as yf

Start='2008-01-01'; End='2015-12-31'; 
asset = ['^GSPC','AAPL' ,'MSFT' ,'NVDA', 'JNJ', 'NVS' ,'JPM' ,'GS' ,'AMZN']
Database=yf.download(asset,Start, End, progress=False).resample('W-FRI').last()['Adj Close']
Database
# compute_summary_statistics(Database)

,AAPL,AMZN,GS,JNJ,JPM,MSFT,NVDA,NVS,^GSPC
Date,,,,,,,,,
2008-01-04,5.458064,4.439500,155.722656,41.469303,27.469221,25.068111,6.880695,27.101755,1411.630005
2008-01-11,5.234953,4.054000,154.795792,42.754219,27.422249,24.725399,6.204092,28.675018,1401.020020
2008-01-18,4.891494,3.988000,145.815247,41.752739,26.569908,24.069166,5.697213,26.483332,1325.189941
2008-01-25,3.941144,3.880000,149.317764,39.340427,29.287975,24.018137,5.722443,24.855644,1330.609985
2008-02-01,4.054521,3.731500,162.121780,39.907269,32.381866,22.202555,6.160515,24.949644,1395.420044
...,...,...,...,...,...,...,...,...,...
2015-12-04,27.126707,33.632000,163.150909,83.646202,54.560051,49.786152,8.237806,57.215141,2091.689941
2015-12-11,25.793503,32.007500,151.618118,82.614334,51.490086,48.138775,7.930264,56.087021,2012.369995
2015-12-18,24.164034,33.207001,150.699265,82.833702,51.755295,48.201111,7.844831,56.252918,2005.550049


In [51]:
Database.to_csv('ADF_test.csv')

In [126]:
compute_summary_statistics(Database).T

NameError: name 'Database' is not defined

In [67]:
import numpy as np

# Redefining the given functions and the example data
def ReturnCalculation(Database, Lag):
    # Dummy return calculation function for the example
    # In a real scenario, this function calculates the returns based on the given lag
    return Database, list(range(len(Database)))

def SDCalculation(DailyReturns, LagSD):
    dimension = len(DailyReturns)
    dif = LagSD
    Out = np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i] = np.std(DailyReturns[i:i+LagSD], ddof=1)
    return np.append(np.repeat(np.nan, dif), Out)

def TrueSDCalculation(DailyReturns, LagSD):
    dimension = len(DailyReturns)
    dif = LagSD
    Out = np.zeros([dimension-dif+1])
    for i in range(dimension-dif+1):
        Out[i] = np.std(DailyReturns[i:i+LagSD], ddof=1)
    return np.append(Out, np.repeat(np.nan, dif-1))

# Example data
DailyReturns = [1, 2, 4.5, 7.3, 3.1, 6.1,3, 7.1, 4.8, 6.9, 10]
LagSD = 3

# Calculations
sd_values = SDCalculation(DailyReturns, LagSD)
true_sd_values = TrueSDCalculation(DailyReturns, LagSD)

sd_values, true_sd_values


(array([       nan,        nan,        nan, 1.80277564, 2.65141472,
        2.13853532, 2.16333077, 1.76162803, 2.13775583, 2.05507502,
        1.27410099]),
 array([1.80277564, 2.65141472, 2.13853532, 2.16333077, 1.76162803,
        2.13775583, 2.05507502, 1.27410099, 2.61597655,        nan,
               nan]))